In [32]:
import cv2
face_cap=cv2.CascadeClassifier("C:/Users/rishi/Downloads/New folder/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml")



video_cap=cv2.VideoCapture(0)
while True:
    ret,video_data=video_cap.read()
    col=cv2.cvtColor(video_data,cv2.COLOR_BGR2GRAY)
    faces=face_cap.detectMultiScale(
        col,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30,30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    for(x,y,w,h) in faces:
        cv2.rectangle(video_data,(x,y),(x+w,y+h),(0,225,0),2)
    cv2.imshow("video_live",video_data)
    h=cv2.waitKey(40)&0xff
    h = cv2.waitKey(1) & 0xFF
    if h == ord('q') or h == 27:
        break
video_cap.release()
cv2.destroyAllWindows()

In [20]:
import cv2
import os

# Create dataset directory if it doesn't exist
dataset_path = r"D:\face recognisition system\images"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

# Load face detection model
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Get employee ID and name
employee_id = input("Enter Employee ID: ")
employee_name = input("Enter Employee Name: ")

# Define file path for the image
employee_image_path = os.path.join(dataset_path, f"{employee_id}_{employee_name}.jpg")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        cv2.imwrite(employee_image_path, face)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        cv2.imshow("Captured Face", frame)
        cv2.waitKey(2000)  # Display the captured image for 2 seconds
        break  # Save only one image

    break  # Exit loop after capturing one face

cap.release()
cv2.destroyAllWindows()

print(f"✅ Employee image saved as {employee_image_path}")

Enter Employee ID:  03
Enter Employee Name:  Vrinda


✅ Employee image saved as D:\face recognisition system\images\03_Vrinda.jpg


In [29]:
import cv2
import os
import numpy as np
import pandas as pd
from datetime import datetime

# Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load dataset
face_database = []
employee_data = {}

face_path = r"D:\face recognisition system\images"

for file in os.listdir(face_path):
    if "_" in file and file.endswith(".png"):
        emp_id, emp_name, _ = file.split("_", 2)
        emp_name = emp_name.rsplit(".", 1)[0]  # Remove file extension
        
        img = cv2.imread(os.path.join(face_path, file), cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        face_database.append((int(emp_id), emp_name, img))
        employee_data[int(emp_id)] = emp_name

# Open webcam
cap = cv2.VideoCapture(0)

# CSV File for attendance
csv_file = "attendance.csv"

# Ensure CSV exists
if not os.path.exists(csv_file):
    pd.DataFrame(columns=["Employee ID", "Employee Name", "Date", "Time"]).to_csv(csv_file, index=False)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        detected_face = gray[y:y + h, x:x + w]  # Extract face region
        detected_face_resized = cv2.resize(detected_face, (100, 100))  # Resize for comparison

        best_match_id = None
        best_match_name = "Unknown"
        min_diff = float("inf")

        # Compare detected face with stored faces
        for emp_id, emp_name, stored_face in face_database:
            stored_face_resized = cv2.resize(stored_face, (100, 100))
            diff = np.sum((detected_face_resized - stored_face_resized) ** 2)

            if diff < min_diff:
                min_diff = diff
                best_match_id = emp_id
                best_match_name = emp_name

        if min_diff < 500000:  # Threshold to allow matching
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{best_match_name} ({best_match_id})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # Log attendance
            now = datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
            df = pd.read_csv(csv_file)
            new_entry = pd.DataFrame([[best_match_id, best_match_name, *now.split(",")]], columns=df.columns)
            df = pd.concat([df, new_entry], ignore_index=True)
            df.to_csv(csv_file, index=False)

            print(f"✅ Access Granted: {best_match_name} ({best_match_id})")
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            print("❌ Access Denied: Unknown Face")

    # Show output
    cv2.imshow("Face Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access Denied: Unknown Face
❌ Access D